# Korean LLM (Large Language Model) Inference
---

### Model: [KKULM-Polyglot-5.8B](https://huggingface.co/nlpai-lab/kullm-polyglot-5.8b-v2)
- LLM GitHub: https://github.com/nlpai-lab/KULLM

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates') 

In [2]:
!pip install -qU boto3 huggingface_hub sagemaker langchain deepspeed 

In [3]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path("/home/ec2-user/SageMaker/hf_cache")
local_model_path.mkdir(exist_ok=True)
model_id = "nlpai-lab/kullm-polyglot-5.8b-v2"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_id,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [4]:
import torch
import deepspeed
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, GPTNeoXLayer

with deepspeed.OnDevice(dtype=torch.float16, device="cuda"):
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        cache_dir=local_model_path
    )

config = {
    "tensor_parallel": {"tp_size": 1},
    "dtype": "fp16",
    "injection_policy": {
        GPTNeoXLayer:('attention.dense', 'mlp.dense_4h_to_h')
    }
}

model = deepspeed.init_inference(model, config)

local_rank = int(os.getenv('LOCAL_RANK', '0'))
tokenizer = AutoTokenizer.from_pretrained(model_id)
generator = pipeline(
    task="text-generation", model=model, tokenizer=tokenizer, device=local_rank
)

[2023-07-23 06:18:51,614] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[2023-07-23 06:19:02,655] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.10.0, git-hash=unknown, git-branch=unknown
[2023-07-23 06:19:02,656] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'InferenceEngine' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusF

In [5]:
import json
from inference_lib import Prompter
prompter = Prompter("kullm")

params = {
    "do_sample": False,
    "max_new_tokens": 256,
    "return_full_text": True,
    "temperature": 0.2,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.5,
    "presence_penalty": None,
    "eos_token_id": 2,
}

instruction = "아래 질문에 100글자 이상으로 자세하게 대답해줘."
input_text = "고려대학교에 대해서 알려줘"
prompt = prompter.generate_prompt(instruction, input_text)
payload = {
    "text_inputs": [prompt,],
    "params": params
}

In [6]:
text_inputs, params = payload["text_inputs"], payload["params"]
result = generator(text_inputs, **params)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[{'generated_text': '고대(고려대)은 한국에서 가장 오래된 대학 중 하나로, 서울특별시 성북구 안암동과 정릉동 사이의 언덕 위에 위치하고 있습니다. 1897년 설립되었으며 현재 약 1만 명의 학생들이 재학중인 종합 대학교이자 연구 중심 대학원 과정도 운영합니다. 고대는 세계적으로 인정받고 존경받아 온 역사적 전통 외에도 다양한 학문 분야 및 프로그램을 보유하여 전세계 학자들에게 영감을 주고 있다는 점 때문에 국제 사회가 주목할 만 합니다. 고려대학은 또한 교육 기관으로서 우수성뿐 아니라 혁신성과 창의성이라는 명성까지 얻기 위해 노력했는데, 이 두 가지 모두 오늘날 고려대의 성공 비결이라고 할 수있다.'}]]
